In [1]:
import torch

In [2]:
# input [batch x frames x input_size]

input = torch.zeros(100, 400, 6)

In [3]:
# spatial embedd 
in_channels = 6
out_channels = 6
spatial_embedder = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm1d(out_channels),
            torch.nn.Linear(400, 400)
            ) #TODO linear layers (batch ve linear layerin cogul eki herhalde 1 tane var)?, sizelari salladim

conv1d = torch.nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
input_t = input.transpose(1, 2).contiguous()

In [4]:
# channel mantiken x,y,z axisleri olan 6 dimensionli taraf olmali
linear = torch.nn.Linear(400, 400)

out = spatial_embedder(input_t)

out.shape[2]




400

In [5]:
from model_temporal import BilinearLSTMSeqNetwork
input = torch.zeros(100, 400, 6).to(0)
a = BilinearLSTMSeqNetwork(6, 6, 100, 0).to(0)
a(input).shape



torch.Size([100, 400, 6])

In [6]:
torch.cuda.is_available()

True

In [7]:

from model_temporal import SpatialEncoderLayer
s = SpatialEncoderLayer(6, 6)

In [8]:
print("input", input_t.shape)
a1 = spatial_embedder(input_t)
print("conv11_0", s.conv11_0(a1).shape)

a2 = s.conv11_0(a1)
print("batch norm 0", s.batch_norm_0(a1).shape)
a3 = s.batch_norm_0(a2)
print("relu", s.relu(a3).shape)
a4  = s.relu(a3)
print("conv33", s.conv33(a4).shape)
c1 = s.conv33(a4)
print("concat", torch.concat([c1, a2], 1).shape)
t = torch.concat([c1, a2], 1)
print("weight", s.conv11_2(t).shape)
t2 = s.conv11_2(t)
print('relu', s.relu(t2).shape)
t3 = s.relu(t2)
v = s.conv11_1(input_t)
print("c2", torch.mul(t3, v).shape)
c2 = torch.mul(t3, v)
print("transpose", c2.transpose(1,2).contiguous().shape)
c3 = c2.transpose(1, 2).contiguous()
c4 = c1.transpose(1, 2).contiguous()
y, _ = s.attention(c4, c3, c3)
print("attention",y.shape)
print("re transpose", y.transpose(1,2).shape)
yt = y.transpose(1,2)
print("conv11_3", s.conv11_3(yt).shape)
z = s.conv11_3(yt)
print('bn 1', s.batch_norm_1(z).shape)
z1 = s.batch_norm_1(z)
print('relu', s.relu(z1).shape)
z2 = s.relu(z1)
z2t = z2.transpose(1,2).contiguous()
y2, _ = s.global_self_attention(z2t, z2t, z2t)
print("global self attention", y2.shape)
y2t = y2.transpose(1,2).contiguous()
print("conv11", s.conv11_4(y2t).shape)
y3 = s.conv11_4(y2t)
y4 = s.batch_norm_2(y3)
y5 = y4 + input_t
y6 = s.relu(y5)
print(y6.shape)
print("layer shape", s.conv11_0.weight.data.shape)


input torch.Size([100, 6, 400])
conv11_0 torch.Size([100, 6, 400])
batch norm 0 torch.Size([100, 6, 400])
relu torch.Size([100, 6, 400])
conv33 torch.Size([100, 6, 400])
concat torch.Size([100, 12, 400])
weight torch.Size([100, 6, 400])
relu torch.Size([100, 6, 400])
c2 torch.Size([100, 6, 400])
transpose torch.Size([100, 400, 6])
attention torch.Size([100, 400, 6])
re transpose torch.Size([100, 6, 400])
conv11_3 torch.Size([100, 6, 400])
bn 1 torch.Size([100, 6, 400])
relu torch.Size([100, 6, 400])
global self attention torch.Size([100, 400, 6])
conv11 torch.Size([100, 6, 400])
torch.Size([100, 6, 400])
layer shape torch.Size([6, 6, 1])


In [9]:
# temporal branch
input2 = torch.zeros(100, 400, 6).to(0)
temp = torch.nn.TransformerDecoderLayer(6, 2)
temp2 = torch.nn.TransformerDecoder(temp, 1).to(0)
p = BilinearLSTMSeqNetwork(6, 6, 100, 0).to(0)

print("temporal embedding ", p(input2).shape)
zz = p(input2)
y6_t = y6.transpose(1, 2).contiguous()
zz_t = zz.transpose(1, 2).contiguous()
print("zz", zz.shape, "y6", y6.shape, "y6_t", y6_t.shape)
print("temporal decoder", temp2(zz, y6_t.to(0)).shape)


temp3 = torch.nn.TransformerDecoderLayer(400, 8)
temp4 = torch.nn.TransformerDecoder(temp3, 1).to(0)
print("zz_t", zz_t.shape, "y6", y6.shape)
print("temporal decoder", temp4(zz_t, y6.to(0)).shape)

temporal embedding  torch.Size([100, 400, 6])
zz torch.Size([100, 400, 6]) y6 torch.Size([100, 6, 400]) y6_t torch.Size([100, 400, 6])
temporal decoder torch.Size([100, 400, 6])
zz_t torch.Size([100, 6, 400]) y6 torch.Size([100, 6, 400])
temporal decoder torch.Size([100, 6, 400])


In [10]:
h = temp4(zz_t, y6.to(0))
ht = h.transpose(1,2).contiguous()

from model_temporal import CTINNetwork

ctin = CTINNetwork(6, 2, 100, 400, 1, 4, 0).to(0)

ctin.linear_linear_vel(ht).shape

torch.Size([100, 400, 2])

In [95]:
out1, out2  = ctin(input)
print(f"input shape {input.shape}, out1 shape {out1.shape}, out2 shape {out2.shape}")
gt = torch.ones(100, 400, 2).to(0)


input shape torch.Size([100, 400, 6]), out1 shape torch.Size([100, 400, 2]), out2 shape torch.Size([100, 400, 2])


In [20]:
pred_pos = torch.cumsum(out1[:, 1:, ], 1)
gt_pos = torch.cumsum(gt[:, 1:, ], 1)
pred_pos.shape

torch.Size([100, 399, 2])

In [62]:
mse_loss = torch.nn.MSELoss()
lv_p = mse_loss(pred_pos, gt_pos)
print("lvp shape", lv_p.shape)
diffs = torch.square(gt[:, 1:, ] - out1[:, 1:, ])
lv_e = torch.cumsum(diffs, 1)
print("cumsum",lv_e.shape)
lv_e = torch.mean(lv_e)
print("mean", lv_e.shape)
lv = lv_p + lv_e
print("lv", lv.shape)


lvp shape torch.Size([])
cumsum torch.Size([100, 399, 2])
mean torch.Size([])
lv torch.Size([])


In [151]:
diff = torch.square(gt - out1)
p1 = (1 / (torch.diag_embed(out2) + 1e-5))
print("diff", diff.shape)
print("p1",p1.shape)
p2 = diff[:, :, None, :]
print("p2", p2.shape)
lc = p2 @ p1
print("lc",lc.shape)
#lc += 0.5 * torch.log(out2[:, :1, 0] * out2[:, :1, 1]) 
lc2  = lc @ diff[:, :, :, None]
print(lc2.shape)
lcf = torch.mean(lc2)
print(lcf.shape)

diff torch.Size([100, 400, 2])
p1 torch.Size([100, 400, 2, 2])
p2 torch.Size([100, 400, 1, 2])
lc torch.Size([100, 400, 1, 2])
torch.Size([100, 400, 1, 1])
torch.Size([])


In [129]:
a = torch.Tensor([[3], [2]])
print("diff", a.shape)
b = torch.Tensor([[1, 0], [0, 2]])
c = torch.Tensor([[1], [2]])
print(a.transpose(0, 1) @ b @ a)
print((a * c).transpose(0, 1) @ a)
print(torch.square(a).transpose(0, 1) @ c)
print(c , 1/c)

kk = torch.zeros(2, 1, 10, 10)
tt = torch.zeros(2, 2, 10, 10)
print((tt @ kk).shape)

aa = torch.zeros(2, 1)
bb = torch.zeros(2, 2)
print((bb @ aa).shape)

yy = torch.zeros(10, 10, 1, 2)
zz = torch.zeros(10, 10, 2, 2)
print((yy @ zz).shape)


diff torch.Size([2, 1])
tensor([[17.]])
tensor([[17.]])
tensor([[17.]])
tensor([[1.],
        [2.]]) tensor([[1.0000],
        [0.5000]])
torch.Size([2, 2, 10, 10])
torch.Size([2, 1])
torch.Size([10, 10, 1, 2])


In [130]:
import torch

a = torch.randn(10, 100, 2)
b = torch.randn(10, 100, 2, 2)

# Perform element-wise matrix multiplication using torch.einsum
result = torch.einsum('...ij,...ijk->...ik', a, b)

print(result.shape)

torch.Size([10, 100, 2])


In [131]:
a = torch.randn(10, 100, 2)
b = torch.randn(10, 100, 2, 2)

result = torch.matmul(a[:, :, None, :], b[:, :, :, :])

print(result.shape)

torch.Size([10, 100, 1, 2])
